Centro de Estudos e Sistemas Avançados do Recife

Pós-graduação em Engenharia e Análise de Dados

Disciplina de RNA e Deep Learning

Professor: Silvan Ferreira da Silva Junior 
 
Grupo:
* Anísio Pereira Batista Filho (apbf@cesar.school)
* Carlos Cezar Lopes de Mendonça (cclm@cesar.school)
* Rodolpho Victor França Valsconcelos (rvfv@cesar.school)

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

## Modelos

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
response = llm.invoke("Olá")

print(response)

content='Olá! Como posso ajudar você hoje?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'stop', 'logprobs': None} id='run-131cada6-b4ac-4cc6-8d2d-f98a11d5c8f4-0' usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16}


## Prompts

### Templates Simples

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: Artificial Intelligence is the future!')]


In [5]:
response = llm.invoke(prompt)

print(response.content)

A inteligência artificial é o futuro!


### Templates de Mensagens

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

# messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

content='Olá, como você está?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 36, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-c006a764-2219-45c2-b017-6eaee4658707-0' usage_metadata={'input_tokens': 36, 'output_tokens': 6, 'total_tokens': 42}


In [7]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [8]:
prompt = prompt_template.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "português",
    "texto": "Hello, how are you?"
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas.'), HumanMessage(content='Hello, how are you?')]


In [9]:
response = llm.invoke(prompt)

print(response.content)

Olá, como você está?


### Parsers

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [11]:
output = parser.invoke(response)

print(output)

Olá, como você está?


## Encadeamento

In [12]:
chain = prompt_template | llm | parser

In [13]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [14]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [15]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

¡Las playas de Recife tienen tiburones!


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [16]:
prompt_template = ChatPromptTemplate.from_template("Faça uma piada sobre {topico}")

chain = prompt_template | llm | parser

chain.invoke({"topico" : "papagaio"})

'Por que o papagaio não usa Facebook?\n\nPorque ele já tem muitos "amigos" e não quer ser "bloqueado"! 🦜😄'

### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [17]:
prompt_template = ChatPromptTemplate.from_template("Classifique o sentimento do seguinte texto em positivo, neutro ou negativo: {texto}")

chain = prompt_template | llm | parser

response = chain.invoke({"texto": "Estou muito feliz hoje!"})
print(response)


O sentimento do texto "Estou muito feliz hoje!" é positivo.


### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um programador python sênior"),
        ("user", "{texto}")
    ]
)

chain = prompt_template | llm | parser

resultado = chain.invoke({"texto" : "faça uma função que multiplique 2 números"})

print(resultado)

Claro! Aqui está uma função simples em Python que multiplica dois números:

```python
def multiplicar(a, b):
    return a * b

# Exemplo de uso
resultado = multiplicar(3, 4)
print(f"O resultado da multiplicação é: {resultado}")
```

Nessa função `multiplicar`, você passa dois argumentos, `a` e `b`, e ela retorna o resultado da multiplicação desses dois números. No exemplo, multiplicamos 3 por 4, resultando em 12. Você pode testar a função com diferentes valores!


### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [19]:
prompt_template_topico = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um especialista em conhecimentos gerais"),
        ("user", "{topic}")
    ]
)

prompt_template_tradutor = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor"),
        ("user", "{explanation}")
    ]
)

chain_1 = prompt_template_topico | llm | parser
chain_2 = prompt_template_tradutor | llm | parser

def explain_and_translate(topic):
    explanation = chain_1.invoke({"topic": topic})

    translation = chain_2.invoke({"explanation": explanation})

    return translation

# Exemplo de uso
resultado = explain_and_translate("O que é a inteligência artificial?")
print(resultado)

Artificial Intelligence (AI) refers to the field of computer science that focuses on creating systems and algorithms capable of performing tasks that typically require human intelligence. This includes skills such as reasoning, learning, perception, natural language understanding, pattern recognition, and decision-making.

AI can be divided into two main categories:

1. **Weak AI (or Narrow AI)**: Systems designed to perform a specific task, such as virtual assistants (like Siri or Google Assistant), recommendation systems, or facial recognition tools. These systems are limited to their specific functions and do not possess consciousness or general understanding.

2. **Strong AI (or General AI)**: A hypothetical form of artificial intelligence that would have the ability to understand, learn, and apply knowledge in a manner similar to a human across a wide range of tasks. This form of AI has not yet been achieved and is a topic of intense research and debate.

AI employs techniques suc

### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [20]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "O Centro de Estudos e Sistemas Avançados do Recife, também conhecido por seu acrônimo CESAR, é um centro de pesquisa e inovação sem fins lucrativos com sede na cidade do Recife, Pernambuco e filiais em Sorocaba, Curitiba e Manaus.[1] O CESAR foi fundado em 1996 por três professores do Centro de Informática da UFPE, Silvio Meira, Fábio Silva e Ismar Kaufman, como forma de aproximar a academia do mercado.Em 2019, o centro conta com mais de 600 funcionários e em 2018 seu faturamento foi da ordem de R$ 100 milhões. O CESAR é parte integrante e instituição âncora do Porto Digital, um dos maiores parques tecnológicos do Brasil também sediado em Recife. Em 2010 o instituto ganhou o prêmio Finep de Inovação na categoria de melhor instituição brasileira de ciência e tecnologia. Área Educacional Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais. O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10] Em 2016 o Ministério da Educação autorizou o CESAR a ofertar cursos de graduação[11] e começaram a ser ofertados os cursos de Ciência da Computação e Design, com abertura da primeira turma no primeiro semestre de 2018.[12] A unidade de ensino superior usa a metodologia de aprendizagem baseada em problemas, que permite o contato com demandas reais da sociedade desde os primeiros dias do curso. Em 2019 o CESAR se tornou a primeira instituição do país a oferecer um curso de doutorado na modalidade profissional na área de engenharia de software. Curso/Programa Graduação Ciência da Computação	Recife	[11] Design	Recife	[11] Mestrado Profissional Engenharia de Software	Recife e Manaus	[8][15] Design de Artefatos Digitais	Recife e Manaus	[16][15] Doutorado Profissional Engenharia de Software	Recife	[4][14] Empreendedorismo e CESAR.labs O instituto mantém uma incubadora de empresas e uma aceleradora, chamada CESAR.labs.[17] O CESAR.labs é uma das aceleradoras parceiras do programa do governo federal Start-up Brasil,[18] tendo acelerado através desse programa 12 empresas.[19] A aceleradora abre chamadas anualmente, investindo até R$ 200 mil por startup.[20] No total, o CESAR já contribuiu para a criação de mais de 50 novas empresas.[3]Empresas incubadas e aceleradas de destaque: Tempest Security Intelligence[21][22] NeuroUp[23] FusionTrak[24] Pitang[25] Radix.com[26] Unidades e Subsidiárias O CESAR possui quatro unidades, sendo sua sede no Recife e três filiais em Manaus, Curitiba e Sorocaba.[27] Além das filiais, o instituto controla duas subsidiárias. O fundo de participação em empresas CESAR.PAR e a desenvolvedora de projetos de software Pitang.[25]"),
        ("user", "{texto}")
    ]
)

chain = prompt_template | llm | parser

resultado = chain.invoke({"texto" : "Quais empresas encubadas no Cesar School?"})

print(resultado)

O CESAR School não é especificamente uma incubadora de empresas, mas faz parte do CESAR, que possui a CESAR.labs, uma incubadora e aceleradora de startups. Dentro do CESAR.labs, diversas empresas foram incubadas e aceleradas, mas não há uma lista específica de empresas incubadas diretamente pela CESAR School.

Algumas das empresas que se destacaram e foram incubadas ou aceleradas pelo CESAR.labs incluem:

1. Tempest Security Intelligence
2. NeuroUp
3. FusionTrak
4. Pitang
5. Radix.com

Se você estiver interessado em informações específicas sobre empresas incubadas ou projetos na CESAR School, recomendo consultar diretamente o site oficial do CESAR ou da CESAR School para obter informações mais detalhadas e atualizadas.
